![](https://images.pexels.com/photos/1167160/pexels-photo-1167160.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260)

# Rotations Euler

Kevin J. Walchko, Phd

28 June 2020

---

# Euler Angles

The fundamental terms most  people understand in 3D space are: roll ($\phi$), pitch ($\theta$), yaw ($\psi$).

![](body-angles.png)

Below is a NED (North, East, Down) frame (*inertial* in yellow) and a Global Hawk (GH) with a body frame (attached to center mass in black) moving within it. Using a sequence of the above terms, we can identify an object the GH sees in body space and transform it NED space via a rotation sequence (or visa versa).

![](body-frame.png)

This diagram comes from the Motion Imagery Standards Board (MISB). Also, the term inertial is *italicized* because the frame is not truely an inertial frame.

## Euler Rotations

Generally, you avoid euler angles and use quaternions to calculate 3D rigid body motion due to gimbol lock. However, quaternions are not intuitive to a human. Thus you use euler angles to visualize what is happening. 

The mathematical transformation between a body fixed frame and an inertial (non-moving) frame can be done with a rotation matrix (not accounting for translation). Euler developed a series of them, with the Aerospace Industry settling on the 3-2-1 sequence to go between an inertial frame and a moving body frame.

Here, the $R_{321} = R_{zyx} = R_x(\phi)R_y(\theta)R_z(\psi)$ is the standard sequence to transform a vector from inertial space to body space. 

$$
\vec{v}^b = R_{321} \vec{v}^I
$$

Aerospace uses a frame attached to a vehicle's center of mass with the x-axis pointing out the nose, y-axis out the right wing, and z-axis pointing down. The local (*inertial*) frame for navigation is NED.

- $R_{321} = R_{NED}$, transform inertial points to body
- Compass is equal to yaw (positive clockwise)
- Gimbal lock at $\pm 90^\circ$ pitch
- Ranges
    - $-180^\circ \leq Roll \leq 180^\circ$
    - $-90^\circ \leq Pitch \leq 90^\circ$
    - $0^\circ \leq Yaw \leq 360^\circ$

The sequence is:

1. Body and inertial frames are aligned
1. Rotate body frame by $\psi$ about z-axis
1. Rotate body frame by $\theta$ about new y'-axis
1. Rotate body frame by $\phi$ about new x''-axis

![](dcm.png)

$$
\vec{p}^b = R_{321} \vec{p}^I = 
\begin{bmatrix}
           c_2c_3 &            c_2s_3 &   -s_2 \\
-c_1s_3+s_1s_2c_3 &  c_1c_3+s_1s_2s_3 & s_1c_2 \\
 s_1s_3+c_1s_2c_3 & -s_1c_3+c_1s_2s_3 & c_1c_2
\end{bmatrix} \vec{p}^I
$$

The simplest case occurs when the inertial and body frames align, then the matix is the identity:

![](identity.png)

Now to calculate the inertial vector from a body frame vector:

$$
\begin{align*}
\vec{p}^I &= R_{123}\vec{p}^b \\
          &= inv(R_{321})\vec{p}^b = R_{321}^T\vec{p}^b
\end{align*}
$$


## Calculating Euler Angles from Body Rates (Gyros)

Given the values $\begin{bmatrix} p & q & r \end{bmatrix}^T$ from a gyro, we want to be able to find the Euler angles ($\Phi =\begin{bmatrix} \phi & \theta & \psi \end{bmatrix}^T$). The relationship between the two are:

![](euler_rotation.png)

This is true when integrating fast and the angle are small values. You end up with the following equations:

$$
\begin{bmatrix}
p \\
q \\
r
\end{bmatrix} = 
\begin{bmatrix}
\dot \phi \\
0 \\
0
\end{bmatrix}^b + R(\phi) \begin{bmatrix}
0 \\
\dot \theta \\
0
\end{bmatrix}^{b'} + R(\phi) R(\theta) \begin{bmatrix}
0 \\
0 \\
\dot \psi
\end{bmatrix}^{b''} = \begin{bmatrix}
1 & 0      & -s\theta \\
0 & c\phi  & s\phi c\theta \\
0 & -s\phi & c\phi c\phi
\end{bmatrix} \begin{bmatrix}
\dot \phi \\
\dot \theta \\
\dot \psi
\end{bmatrix} = L^B_I \dot \Phi
$$

where $R$ is direct cosine matrix (DCM). Now $L^{-1} \neq L^T$ since this is not a symetric matrix. However, this can be rearranged to get:

$$
\dot \Phi =
\begin{bmatrix}
\dot \phi \\
\dot \theta \\
\dot \psi
\end{bmatrix} = 
\begin{bmatrix}
1 & s\phi t\theta & c\phi t\theta \\
0 & c\phi & -s\phi \\
0 & s\phi sec\theta & c\phi sec\theta
\end{bmatrix} \begin{bmatrix}
p \\
q \\
r
\end{bmatrix} = L^I_B \omega_B
$$

Now you could integrate this equation in order to update the Euler angles given gyro measurements ($\Phi \mathrel{{+}{=}} \omega_B \Delta t$). However, Euler angles have issues (i.e., gimbal lock) so typically engineers use quaternions instead to track the orientation of a rigid body. However, since quaternions are a 4D representation of 3D space, they are highly non-intuitive. Therefore, the quaternion is typically transformed back into Euler angles for human understanding only.

## References

- Wikipedia: [Aircraft principle axes](https://en.wikipedia.org/wiki/Aircraft_principal_axes)
- Wikipedia: [Axis Convention](https://en.wikipedia.org/wiki/Axes_conventions)
- Wikipedia: [Euler Angles](https://en.wikipedia.org/wiki/Euler_angles)
- Wikipedia: [Transpose of matricies: $(AB)^T = B^TA^T$](https://en.wikipedia.org/wiki/Transpose)
- [Explanation of Euler Rotations](https://charlestytler.com/measuring-quadcopter-attitude-sensors/)
- Scipy: [Rotation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html)

In [1]:
%load_ext autoreload
%autoreload 2

from math import sin, cos, tan, acos, pi
import numpy as np

# Derivative function
def xdot(x,u):
    x_dot = np.zeros(3)  # [0,0,0]
    #   phidot = p + (q*sin(phi) + r*cos(phi))*tan(theta)
    x_dot[0] = u[0] + (u[1]*sin(x[0]) + u[2]*cos(x[0]))*tan(x[1])
    #   thetadot = q*cos(phi) - r*sin(phi)
    x_dot[1] = u[1]*cos(x[0]) - u[2]*sin(x[0])
    #   psidot = (q*sin(phi) + r*cos(phi))*sec(theta)
    x_dot[2] = (u[1]*sin(x[0]) + u[2]*cos(x[0]))*acos(x[1])
    return x_dot

# Initial condition setup
eulerAngles = np.zeros(3)  # Assume start up with phi = theta = psi = 0 radians
deltaT = 0.01              # Sampling time (sec)

# loop
for i in range(10):
    omegaRead = [0.01,0.1,0.1] # gyros.getAngularVelocity()
    eulerAngles += xdot(eulerAngles,omegaRead) * deltaT
    
print(f"{np.rad2deg(eulerAngles)} deg")

[0.05987511 0.572693   0.89783636] deg


# Scipy

`scipy` has a built in euler/quaternion/matrix capability. Just be **aware** that the order doesn't match:

- `scipy` is (imaginary, real): `q = [x,y,z,w]`
- `squaternion` is (real, imaginary):  `q = [w,x,y,z]`

```
class Quaternion(builtins.object)
 |  Quaternion(w=1.0, x=0.0, y=0.0, z=0.0) -> None
 |  
 |  q = Quaternion() # defaults to unit (1,0,0,0)
 |  q = Quaternion(w,x,y,z)
 |  
 |  class properties:
 |      q.scalar: q.w
 |      q.vector: (q.x, q.y, q.z)
 |      q.normalize: return a normalized quaternion
 |      q.magnitude: returns the magnitude of the quaternion
 |  
 |  class methods:
 |      q.to_euler: returns (roll, pitch, yaw)
 |      Quaternion.from_euler: returns a quaternion from euler angles (Z,Y,X)
 |      q*q: multiply 2 quaternions together
```

In [95]:
from scipy.spatial.transform import Rotation as R
from squaternion import Quaternion

In [101]:
q = Quaternion(np.cos(np.pi/4), 0, np.sin(np.pi/4),1).normalize
w,x,y,z = q
r = R.from_quat([x,y,z,w])
rq = r.as_quat()
print(f"{(rq[3], *rq[:3])} == {q}")

(0.5000000000000001, 0.0, 0.5, 0.7071067811865476) == Quaternion(w=0.5, x=0.0, y=0.49999999999999994, z=0.7071067811865475)


In [102]:
print(f"{r.as_euler('zyx', degrees=True)} == {q.to_euler(degrees=True)}")

[125.26438968  30.         -54.73561032] == (54.73561031724534, 29.999999999999996, 125.26438968275463)


In [97]:
r = R.from_euler('zyx',[0,0,0])
q = Quaternion.from_euler(0,0,0)
print(f"{r.as_quat()} == {q}")

[0. 0. 0. 1.] == Quaternion(w=1.0, x=0.0, y=0.0, z=0.0)


# Rotation

```
R321(a,b,c) -> R1(c)R2(b)R3(a)
R313(a,b,c) -> R3(c)R1(b)R3(a)
```

In [81]:
from rotations import R1,R2,R3,R321,R123,R313

In [82]:
print(R1(45,True),"\n") # x
print(R2(45,True),"\n") # y
print(R3(45,True),"\n") # z

[[ 1.          0.          0.        ]
 [ 0.          0.70710678  0.70710678]
 [ 0.         -0.70710678  0.70710678]] 

[[ 0.70710678  0.         -0.70710678]
 [ 0.          1.          0.        ]
 [ 0.70710678  0.          0.70710678]] 

[[ 0.70710678  0.70710678  0.        ]
 [-0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]] 



In [83]:
print("R321:\n", R1(60,True) @ R2(40,True) @ R3(20,True))

R321:
 [[ 0.71984631  0.26200263 -0.64278761]
 [ 0.35208899  0.6602388   0.66341395]
 [ 0.59820952 -0.70387453  0.38302222]]


In [84]:
print("R321:\n", R321(20,40,60,True))

R321:
 [[ 0.71984631  0.26200263 -0.64278761]
 [ 0.35208899  0.6602388   0.66341395]
 [ 0.59820952 -0.70387453  0.38302222]]


In [85]:
r321 = R1(60,True) @ R2(40,True) @ R3(20,True)
np.allclose(r321, R321(20,40,60,True))

True

In [86]:
r313 = R3(20, True) @ R1(40,True) @ R3(60,True)
np.allclose(r313, R313(60,40,20,True))

True

## Inverse

So we know this:

$$
R(a)^{-1} = R(a)^T \\
(AB)^T = B^TA^T
$$

But let's show the details a little more to prove it by moving individual matrices to the other side of the equation:

$$
\begin{align*}
                                 A &= R_{321}(a,b,c) B \\
                                 A &= R_{1}(c) R_{2}(b) R_{3}(a) B \\
R_{3}(a)^T R_{2}(b)^T R_{1}(c)^T A &= B \\
                  R_{123}(c,b,a) A &= B
\end{align*}
$$

or we could use the transpose property above ($(AB)^T = B^TA^T$):

$$
\begin{align*}
                                 A &= R_{321}(a,b,c) B \\
                R_{321}(a,b,c)^T A &= B \\
  [R_{1}(c) R_{2}(b) R_{3}(a)]^T A &= B \\
R_{3}(a)^T R_{2}(b)^T R_{1}(c)^T A &= B \\
                  R_{123}(c,b,a) A &= B
\end{align*}
$$

Notice how the inverse of $R_{321}(a,b,c)$ is $R_{123}(c,b,a)$ with the values swapped around.
Thus, from above, we can state:

$$
\begin{align*}
R_{321}(a,b,c)^{-1} &= R_{321}(a,b,c)^T \\
             &= R_{3}(a)^T R_{2}(b)^T R_{1}(c)^T \\
             &= R_{123}(c,b,a)
\end{align*}
$$

In [87]:
# R321(c,b,a) == R123(a,b,c).T
#             == [R3(c)R2(b)R1(a)].T
#             == R1(a).T R2(b).T R3(c).T
#             == R321(c,b,a)
np.allclose(R321(20,40,60,True), R123(20,40,60,True).T)

True

In [88]:
rt123 = R3(20,True).T @ R2(40,True).T @ R1(60,True).T
np.allclose(rt123, R123(20,40,60,True))

True

In [89]:
np.allclose(rt123, R321(20,40,60,True).T)

True